In [2]:
import onnx
from onnx.helper import *
import numpy as np
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator

## Make node

In [3]:
import torch

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from torch import nn as nn

In [3]:
 m = nn.Conv2d(32, 32 , 3, stride=2,padding = 1, groups = 32)

In [4]:
data = torch.randn(1, 32, 104, 104)

In [5]:
output = m(data)

In [32]:
torch.onnx.export(m,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/code/CodeReading/Paddle-Lite/pnn/conv_depthwise.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

In [6]:
output.shape

torch.Size([1, 32, 52, 52])

In [7]:
m.bias

Parameter containing:
tensor([-0.2141,  0.3037, -0.0095, -0.2251, -0.2738,  0.3105,  0.0239, -0.2920,
        -0.3256, -0.2038, -0.2915, -0.1269, -0.0550,  0.2707, -0.3201, -0.3210,
        -0.3223, -0.0352, -0.2593, -0.0653, -0.1905, -0.3320, -0.2805, -0.2138,
        -0.3333, -0.1204,  0.2564,  0.1934, -0.0482, -0.2368, -0.1808, -0.2981],
       requires_grad=True)

In [8]:
weight = m.weight.detach()

In [9]:
pnn_weight = weight.permute(0,2,3,1)

In [10]:
mn_weight = pnn_weight.reshape([5,12])

RuntimeError: shape '[5, 12]' is invalid for input of size 288

In [ ]:
# reshape 到m * n 并transpose
weight = torch.transpose(weight.reshape([5,12]),0,1)

In [39]:
weight

tensor([[-0.1520,  0.2060,  0.2284, -0.1810,  0.2607],
        [ 0.0295, -0.0378,  0.0138,  0.2794,  0.1455],
        [ 0.0082, -0.2350,  0.1256,  0.0217,  0.1567],
        [ 0.0489, -0.0637,  0.1744,  0.1335,  0.0948],
        [-0.0311,  0.2040, -0.1192, -0.1462, -0.0013],
        [-0.0947,  0.1130, -0.1895, -0.0385,  0.0461],
        [ 0.1862, -0.0235, -0.2529,  0.2474, -0.1296],
        [-0.0883,  0.1721,  0.1799,  0.1601, -0.1431],
        [ 0.0591, -0.1370,  0.2683, -0.0775,  0.1635],
        [ 0.1586, -0.1273, -0.0922, -0.1727,  0.0998],
        [ 0.1335,  0.0992,  0.0145, -0.1227,  0.2118],
        [-0.1693,  0.1448,  0.0321,  0.2753, -0.0215]])

In [55]:
mn_weight

tensor([[-0.1520, -0.0311,  0.0591,  0.0295, -0.0947,  0.1586,  0.0082,  0.1862,
          0.1335,  0.0489, -0.0883, -0.1693],
        [ 0.2060,  0.2040, -0.1370, -0.0378,  0.1130, -0.1273, -0.2350, -0.0235,
          0.0992, -0.0637,  0.1721,  0.1448],
        [ 0.2284, -0.1192,  0.2683,  0.0138, -0.1895, -0.0922,  0.1256, -0.2529,
          0.0145,  0.1744,  0.1799,  0.0321],
        [-0.1810, -0.1462, -0.0775,  0.2794, -0.0385, -0.1727,  0.0217,  0.2474,
         -0.1227,  0.1335,  0.1601,  0.2753],
        [ 0.2607, -0.0013,  0.1635,  0.1455,  0.0461,  0.0998,  0.1567, -0.1296,
          0.2118,  0.0948, -0.1431, -0.0215]])

### Create Model

In [22]:
import onnx
from onnx import helper as oh

In [ ]:
node = oh.make_node('ConvTranspose',inputs='data',outputs=['output'],name='ct')

In [23]:
model = onnx.load("/Users/bob/code/CodeReading/ppl.nn/cmake-build-debug/samples/cpp/run_model/conv_transpose.onnx")

In [25]:
graph = model.graph

In [35]:
del graph.initializer[0].dims[:]

In [36]:
[33,16,3,3]

[33, 16, 3, 3]

In [39]:
graph.initializer[0].dims.extend([33,16,3,3])

In [42]:
onnx.save_model(model,"/Users/bob/code/CodeReading/ppl.nn/cmake-build-debug/samples/cpp/run_model/modified.onnx")

### Conv2DShapeinfer

In [16]:
import tensorflow as tf

In [9]:
output = tf.nn.conv2d(
    tf.random.uniform([1,9,9,3]), # input
    tf.random.uniform([1,1,3,2]), #  filter, hwio ,tf
    strides=[3,3], # strides
    padding = "SAME", # padding
    dilations = None
)

In [10]:
output.shape

TensorShape([1, 3, 3, 2])

In [69]:
ct = tf.nn.conv_transpose(
    tf.random.uniform([1,3,3,2]),
    tf.random.uniform([2,2,3,2]),
    [1,7,7,3],
    strides=[3,3],
    padding = "SAME", # padding
    dilations = None
)

In [70]:
ct.shape

TensorShape([1, 7, 7, 3])

### InputGradTest

In [14]:
def get_data(shape, begin = 0, truncate = False, truncatBase =13):
    # np.array().astype(np.float32).reshape(data_shape)
    size = reduce(operator.mul, shape, 1)
    data = []
    for i in range(size):
        if not truncate:
            data.append(i + begin);
        else:
            tmp =  (i + begin) if (i + begin) < 0 else (i + begin) % truncateBase;
            data.append(tmp)
    return np.array(data).astype(np.float32).reshape(shape)

In [7]:
kernel_transpose = np.array([0,1,2,3,0,1],dtype=np.float32).reshape([2,1,1,3])

In [8]:
kernel_transpose

array([[[[0., 1., 2.]]],


       [[[3., 0., 1.]]]], dtype=float32)

In [9]:
kernel_transpose = np.transpose(kernel_transpose,[1,2,3,0])

In [11]:
kernel_transpose

array([[[[0., 3.],
         [1., 0.],
         [2., 1.]]]], dtype=float32)

In [12]:
kernel_transpose.shape

(1, 1, 3, 2)

In [13]:
output_grad = np.array([   0,
   1,
   2,
   3,
   4,
   0,
   1,
   2,
   3,
   4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4,
  0,
  1,
  2,
  3,
  4],dtype=np.float32).reshape([1,9,5,2])

In [14]:
output_grad.shape

(1, 9, 5, 2)

In [17]:
test_output = tf.nn.conv_transpose(
    output_grad,
    kernel_transpose,
    [1,9,9,3],
    strides=[1,2],
    padding = "SAME", # padding
    dilations = None
)

2022-09-16 18:46:04.637762: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 18:46:04.756624: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7feb86ba65a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-16 18:46:04.756640: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [ ]:
tmp = tf.

In [18]:
kernel_transpose.shape

(1, 1, 3, 2)

In [19]:
kernel_transpose

array([[[[0., 3.],
         [1., 0.],
         [2., 1.]]]], dtype=float32)

In [247]:
test_output.numpy().shape

(1, 9, 9, 3)

In [122]:
test_output.numpy().flatten()[78]

12.0

### TorchWithBias

In [1]:
from torch import nn

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
with_bias = nn.ConvTranspose2d(2, 3, 2, stride=2, )

In [4]:
conv_inp1 = torch.rand(1,2,7,7)
conv_inp2 = torch.rand(1,2,8,8)

conv1 = torch.nn.Conv2d(2, 4, kernel_size = 3, stride = 2)

In [145]:
out1 = conv1(conv_inp1)     
out2 = conv1(conv_inp2)

In [165]:
out2.shape

torch.Size([1, 4, 3, 3])

In [193]:
conv_t1 = torch.nn.ConvTranspose2d(4, 2, kernel_size=3, stride=2)

In [149]:
conv_t2 = torch.nn.ConvTranspose2d(4, 2, kernel_size=3, stride=2, output_padding=1)


In [150]:
transposed2 = conv_t2(out2)

In [151]:
conv_t2.bias

Parameter containing:
tensor([ 0.0806, -0.0169], requires_grad=True)

In [152]:
conv_t2.weight.shape

torch.Size([4, 2, 3, 3])

In [156]:
def torch_weight_to_pnn(weight):
    res = weight.detach()
    return res.permute([0,2,3,1]).numpy()

In [241]:
torch_weight_to_pnn(conv_t2.weight).flatten()

array([-0.08598892,  0.22859292,  0.0242203 , -0.22505076, -0.19747889,
       -0.16396557, -0.13732672, -0.16988268, -0.19650199, -0.11784714,
       -0.09573036, -0.09520026,  0.20134456, -0.0902171 , -0.07027142,
        0.12464471,  0.02211244, -0.14829227, -0.13545354, -0.10666129,
        0.16517378,  0.16190787, -0.22098807,  0.04119961,  0.1871757 ,
       -0.18650287,  0.10129328,  0.01875649,  0.18923567,  0.00457549,
        0.07432802,  0.08531667, -0.08495605,  0.04637216,  0.18991925,
       -0.06346714, -0.18734127, -0.01229569, -0.05700004,  0.01039293,
       -0.03600515,  0.2074878 , -0.23093118, -0.02857821, -0.2168017 ,
        0.1478384 , -0.05225842,  0.0383354 ,  0.10759346,  0.06498723,
       -0.20631063, -0.02114594,  0.17599969, -0.10371006,  0.06678195,
        0.22403146, -0.09102502, -0.15165281, -0.17513567, -0.16076994,
        0.20458682, -0.17611957, -0.11416935,  0.12211417,  0.18790372,
        0.0197589 , -0.16182375,  0.17027919, -0.22771728, -0.23

In [180]:
bias_input = torch.from_numpy(np.array(get_data(1*4*3*3),dtype=np.float32).reshape(1,4,3,3))

In [182]:
bias_input.shape

torch.Size([1, 4, 3, 3])

In [183]:
test_output = conv_t2(bias_input)

In [188]:
test_output.shape

torch.Size([1, 2, 8, 8])

In [240]:
torch_weight_to_pnn(conv_t1.weight).flatten()

array([-0.07447739, -0.13594478, -0.16773763,  0.23172377,  0.15555386,
        0.01324239, -0.21690345, -0.19830418,  0.21050198,  0.15979357,
        0.20807673, -0.03992857,  0.10459785, -0.1875741 , -0.16358006,
       -0.22946231,  0.0553066 ,  0.04950316,  0.11889558,  0.21686982,
        0.13792156,  0.00949842,  0.16429333, -0.081011  , -0.10801502,
        0.09868048, -0.0355915 ,  0.21629916,  0.04159497, -0.06480502,
        0.23078342,  0.14525475,  0.03220336, -0.00782782,  0.11145274,
        0.21582536,  0.01092982, -0.02478327,  0.05318256, -0.1060877 ,
       -0.1375268 ,  0.22525497, -0.06433952,  0.07188036,  0.17733468,
        0.03209467,  0.14347185,  0.19502284, -0.22364798,  0.1730703 ,
       -0.11489874, -0.14852381, -0.00231485, -0.13392021,  0.1441349 ,
        0.06693746,  0.19474347,  0.15590934, -0.02650508, -0.2162298 ,
        0.08019136,  0.15587641, -0.09580086,  0.142491  , -0.05759637,
        0.18616466, -0.06096216, -0.1799687 , -0.0028241 , -0.16

In [196]:
output_2 = conv_t1(bias_input)

In [199]:
output_2.shape

torch.Size([1, 2, 7, 7])

In [200]:
conv_t1.bias

Parameter containing:
tensor([-0.1756, -0.1742], requires_grad=True)

### Tensorflow

#### Case1_Pad_SAME

In [208]:
data_shape = [1,9,9,3]

In [218]:
data = np.array(get_data(reduce(operator.mul, data_shape, 1))).astype(np.float32).reshape(data_shape)

In [215]:
filter_shape = [2,2,3,5]

In [221]:
weight = np.array(get_data(reduce(operator.mul, filter_shape, 1))).astype(np.float32).reshape(filter_shape)

In [268]:
conv_output = tf.nn.conv2d(tf.convert_to_tensor(data), tf.convert_to_tensor(weight), strides=1,dilations=2,padding='SAME')

In [269]:
conv_output.shape

TensorShape([1, 9, 9, 5])

In [270]:
#
t_data = np.array(get_data(reduce(operator.mul, conv_output.shape, 1))).astype(np.float32).reshape(conv_output.shape)

In [272]:
output = tf.nn.conv2d_transpose(t_data,weight,[1,9,9,3],strides=1,dilations=2,padding='SAME')

InvalidArgumentError: Current libxsmm and customized CPU implementations do not yet support dilation rates larger than 1. [Op:Conv2DBackpropInput]

In [255]:
output.shape

TensorShape([1, 9, 9, 3])

In [256]:
output.numpy().flatten()

array([   30.,    80.,   130.,   180.,   230.,   280.,    80.,   255.,
         430.,   605.,   780.,   955.,   130.,   430.,   730.,  1030.,
        1330.,  1630.,   180.,   605.,  1030.,  1455.,  1880.,  2305.,
         230.,   780.,  1330.,   330.,   380.,   430.,   480.,   530.,
         580.,  1130.,  1305.,  1480.,  1655.,  1830.,  2005.,  1930.,
        2230.,  2530.,  2830.,  3130.,  3430.,  2730.,  3155.,  3580.,
        4005.,  4430.,  4855.,  3530.,  4080.,  4630.,   280.,   955.,
        1630.,  2305.,  2980.,  3655.,   330.,  1130.,  1930.,  2730.,
        3530.,  4330.,   380.,  1305.,  2230.,  3155.,  4080.,  5005.,
         430.,  1480.,  2530.,  3580.,  4630.,  5680.,   480.,  1655.,
        2830.,  4330.,  5005.,  5680.,  6355.,  7030.,  7705.,  5130.,
        5930.,  6730.,  7530.,  8330.,  9130.,  5930.,  6855.,  7780.,
        8705.,  9630., 10555.,  6730.,  7780.,  8830.,  9880., 10930.,
       11980.,  7530.,  8705.,  9880.,   530.,  1830.,  3130.,  4430.,
      

In [258]:
weight.transpose([3,0,1,2]).flatten()

array([ 0.,  5., 10., 15., 20., 25., 30., 35., 40., 45., 50., 55.,  1.,
        6., 11., 16., 21., 26., 31., 36., 41., 46., 51., 56.,  2.,  7.,
       12., 17., 22., 27., 32., 37., 42., 47., 52., 57.,  3.,  8., 13.,
       18., 23., 28., 33., 38., 43., 48., 53., 58.,  4.,  9., 14., 19.,
       24., 29., 34., 39., 44., 49., 54., 59.], dtype=float32)

In [249]:
weight.shape

(2, 2, 3, 5)

### Torch dilation 2

In [312]:
nchw_input = get_data([1,3,8,8])

In [329]:
conv = torch.nn.Conv2d(3, 4, kernel_size = [2,1], stride = [1,1],dilation=1,bias=False)

In [330]:
conv_output = conv(torch.from_numpy(nchw_input))

In [331]:
conv_output.shape

torch.Size([1, 4, 7, 8])

In [332]:
t_input = get_data(list(conv_output.shape))

In [334]:
conv_t = torch.nn.ConvTranspose2d(4,3,kernel_size=[2,1],stride=[1,1],dilation=1,bias=False)

In [335]:
output = conv_t(torch.from_numpy(t_input))

In [336]:
output.detach().numpy().shape

(1, 3, 8, 8)

In [337]:
torch_weight_to_pnn(conv_t.weight).flatten()

array([ 0.28062713,  0.30849963,  0.39423364,  0.14346242, -0.35381427,
       -0.06700224,  0.11777908, -0.00255701,  0.29896873,  0.12235141,
        0.32177818,  0.15091985,  0.00405869,  0.3991269 ,  0.34511155,
        0.2601741 , -0.03359014,  0.0280278 , -0.07241759,  0.12268496,
        0.13441616, -0.34851736, -0.28691405, -0.19857115], dtype=float32)

In [338]:
t_input.shape

(1, 4, 7, 8)

In [339]:
output.detach().numpy().flatten()

array([ -5.1159534,  -4.785906 ,  -4.4558587,  -4.1258116,  -3.795764 ,
        -3.4657168,  -3.1356685,  -2.8056223, -25.035315 , -24.527794 ,
       -24.02028  , -23.51276  , -23.00524  , -22.497725 , -21.990206 ,
       -21.482689 , -20.97517  , -20.467653 , -19.960133 , -19.452616 ,
       -18.945099 , -18.43758  , -17.930065 , -17.422544 , -16.91503  ,
       -16.40751  , -15.899991 , -15.392475 , -14.884956 , -14.3774395,
       -13.869921 , -13.362402 , -12.854884 , -12.347364 , -11.839849 ,
       -11.332328 , -10.824816 , -10.317295 ,  -9.809777 ,  -9.302259 ,
        -8.794743 ,  -8.287225 ,  -7.779705 ,  -7.27219  ,  -6.7646704,
        -6.2571507,  -5.7496347,  -5.242114 ,  -4.7346   ,  -4.2270794,
        -3.7195606,  -3.2120447,  -2.704526 ,  -2.197011 ,  -1.6894913,
        -1.1819754, -14.041149 , -13.863682 , -13.686211 , -13.508739 ,
       -13.331268 , -13.153797 , -12.97633  , -12.798859 ,  65.17009  ,
        65.99785  ,  66.8256   ,  67.65336  ,  68.48111  ,  69.3

In [310]:
conv_t.bias

Parameter containing:
tensor([ 0.2103, -0.0517, -0.2749], requires_grad=True)

### Add

In [4]:
add_1 = torch.rand([1,5,1])

In [10]:
add_1.numpy().flatten()

array([0.4863451 , 0.5473978 , 0.5295979 , 0.99181163, 0.83747786],
      dtype=float32)

In [7]:
add_2 = torch.rand([1,5,1])

In [11]:
add_2.numpy().flatten()

array([0.44318914, 0.26870042, 0.37088585, 0.4972793 , 0.82523   ],
      dtype=float32)

In [13]:
(add_1 + add_2).numpy().flatten()

array([0.92953426, 0.8160982 , 0.9004837 , 1.4890909 , 1.6627078 ],
      dtype=float32)

In [14]:
foo = np.array([[1,2,3], [4,5,6]])

In [30]:
tmp = np.ascontiguousarray(foo.transpose([1,0]))

In [31]:
tmp.flags['C_CONTIGUOUS']

True

In [32]:
foo.flags['C_CONTIGUOUS']

True

In [19]:
a = np.array([i for i in range(1,2*3*3*4 + 1)]).reshape((2,3,3,4))

In [20]:
a.flatten()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72])

In [16]:
a = np.transpose(a, (3,1,2,0))

In [17]:
a

array([[[[ 1, 37],
         [ 5, 41],
         [ 9, 45]],

        [[13, 49],
         [17, 53],
         [21, 57]],

        [[25, 61],
         [29, 65],
         [33, 69]]],


       [[[ 2, 38],
         [ 6, 42],
         [10, 46]],

        [[14, 50],
         [18, 54],
         [22, 58]],

        [[26, 62],
         [30, 66],
         [34, 70]]],


       [[[ 3, 39],
         [ 7, 43],
         [11, 47]],

        [[15, 51],
         [19, 55],
         [23, 59]],

        [[27, 63],
         [31, 67],
         [35, 71]]],


       [[[ 4, 40],
         [ 8, 44],
         [12, 48]],

        [[16, 52],
         [20, 56],
         [24, 60]],

        [[28, 64],
         [32, 68],
         [36, 72]]]])

In [1]:
import torch

/opt/anaconda3/envs/d2l/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 5x5s1

In [75]:

model = torch.nn.Conv2d(32,4,(5,5),stride=1,padding=1)

In [76]:
model

Conv2d(32, 4, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))

In [47]:
data = torch.randn(1, 32, 416, 416)

In [48]:
output = model(data)

In [49]:
torch.onnx.export(model,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/docs/ByteDance/PNN/converter/dwm/dwm_5x5s1.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

### 3x3s1

In [77]:
model = torch.nn.Conv2d(32,4,(3,3),stride=1,padding=1)

In [78]:
data = torch.randn(1, 32, 416, 416)

In [79]:
output = model(data)

In [80]:
torch.onnx.export(model,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/docs/ByteDance/PNN/converter/dwm/dwm_3x3s1.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

### 5x5s1s1-14x14

In [2]:
small_model = torch.nn.Conv2d(256,256,(5,5),stride=1,padding=1)

In [3]:
data = torch.randn(1, 256, 14, 14)

In [4]:
output = small_model(data)

In [7]:
torch.onnx.export(small_model,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/docs/ByteDance/PNN/converter/dwm/dwm_5x5s1_small.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )

### 7x7s1

In [5]:
data = torch.randn(1, 32, 416, 416)

In [6]:

model = torch.nn.Conv2d(32,4,(7,7),stride=1,padding=1)

In [8]:
output = model(data)

In [9]:
torch.onnx.export(model,               # model being run
                  data,                         # model input (or a tuple for multiple inputs)
                  "/Users/bob/docs/ByteDance/PNN/converter/dwm/dwm_7x7s1.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['data'],   # the model's input names
                  output_names = ['output'], # the model's output names
                 )